In [4]:
import spacy

2022-12-01 15:28:53.333361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 15:28:53.427916: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-01 15:28:53.834618: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-01 15:28:53.834684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [5]:
spacy_en = spacy.load("en_core_web_sm")
spacy_de = spacy.load("nl_core_news_sm")

In [6]:
tokenized=spacy_en.tokenizer('I am a student')
for i, token in enumerate(tokenized):
    print(f'index{i}:{token.text}')

index0:I
index1:am
index2:a
index3:student


In [7]:
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

In [8]:
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator #torch==1.8.0 torchtext==0.9.0

In [12]:
SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

In [20]:
from torchtext.legacy.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████████████████████████████████████████████████████████| 1.21M/1.21M [00:02<00:00, 488kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|███████████████████████████████████████████████████████████| 46.3k/46.3k [00:00<00:00, 85.0kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|████████████████████████████████████████████████████| 66.2k/66.2k [00:00<00:00, 120kB/s]


In [21]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [22]:
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [23]:
#To get the initial input dimension
SRC.build_vocab(train_dataset, min_freq=2) #select the words appear at least more than 2 
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7853
len(TRG): 5893


In [25]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0 #with stoi, it can check inside 
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # : 2
print(TRG.vocab.stoi["<eos>"]) # : 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


In [26]:
import torch

BATCH_SIZE=128

In [30]:
# BucketIterator : The words contained in one sentence must be entered in order. 
# So it's good to make the number of words in a single batch have similar numbers.
# Input as little padding in sort sentences as possible --> reduce the input dimension

BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator=BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset), batch_size=BATCH_SIZE)


In [29]:
for i, batch in enumerate(train_iterator):
    src=batch.src
    trg=batch.trg
    
    print(f"첫 번째 배치 크기: {src.shape}")
    
    #output of one sentence in the current batch
    for i in range(src.shape[1]): #src.shape[1] : sequence length
        print(f"인덱스 {i}: {src[0][i].item()}") #first sentence
    break 
    #2 is <sos>, 3 is <eos>, the data between them represent each words, 1 is meaningless tocken(padding tocken, after 3)

첫 번째 배치 크기: torch.Size([128, 27])
인덱스 0: 2
인덱스 1: 8
인덱스 2: 16
인덱스 3: 181
인덱스 4: 12
인덱스 5: 5
인덱스 6: 1290
인덱스 7: 126
인덱스 8: 14
인덱스 9: 0
인덱스 10: 4
인덱스 11: 3
인덱스 12: 1
인덱스 13: 1
인덱스 14: 1
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1


In [31]:
import torch.nn as nn

In [78]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio):
        super().__init__()
        
        assert hidden_dim%n_heads==0
        
        self.hidden_dim=hidden_dim # embedding dimension of one word
        self.n_heads=n_heads # number of each attention
        self.head_dim=hidden_dim % n_heads
        
        self.fc_q=nn.Linear(hidden_dim, hidden_dim)
        self.fc_k=nn.Linear(hidden_dim, hidden_dim)
        self.fc_v=nn.Linear(hidden_dim, hidden_dim)
        self.fc_o=nn.Linear(hidden_dim, hidden_dim) #output linear
        self.dropout=nn.Dropout(dropout_ratio)
        self.scale=torch.sqrt(torch.FloatTensor([self.head_dim]))
        
    def forward(self, query, key, value, mask=None):
        batch_size=query.shape[0]
        
        Q=self.fc_q(query)
        K=self.fc_k(query)
        V=self.fc_v(query)
        
        Q=Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3) #reshape
        K=K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V=V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        energy=torch.matmul(Q, K.permute(0,1,3,2))/self.scale #Q * K and divide by scale in each heads
        
        if mast is not None:
            energy=energy.masked_fill(mask==0, -1e9) #fill with -1e9 if mask is 0
        
        attention=torch.softmax(energy, dim=-1)
        
        #scaled dot product attention
        x=torch.matmul(self.dropout(attention),V)
        
        x=x.permute(0,2,1,3).contiguous()
        
        x=x.view(batch_size, -1, self.hidden_dim) #concat
        
        x=self.fc_o(x) 
        
        return x, attention

In [79]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()
        
        self.fc_1=nn.Linear(hidden_dim, pf_dim)
        self.fc_2=nn.Linear(pf_dim, hidden_dim)
        self.dropout=nn.Dropout(dropout_ratio)
        
    def forward(self, x):
        x=self.dropout(torch.relu(self.fc_1(x)))
        x=self.fc_2(x)
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio):
        super().__init__()
        self.self_attn_layer_norm=nn.LayerNorm(hidden_dim)
        self.ff_layer_norm=nn.LayerNorm(hidden_dim)
        self.self_attention=MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio)
        self.positionwise_feedforward=PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout=nn.Dropout(dropout_ratio)
        
    def forward(self, src, src_mask):
        _src, _=self.self_attention(src, src, src, src_mask) #Q,K,V
        src=self.self_attn_layer_norm(src+self.dropout(_src)) #residual connection and then apply to layer normalization
        _src=self.positionwise_feedforward(src) #through feedforward
        src=self.ff_layer_norm(src+self.dropout(_src)) #one more residual connection and then apply to layer normalization
        return src

In [80]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, max_length=100):
        super().__init__()
        self.tok_embedding=nn.Embedding(input_dim, hidden_dim)
        self.pos_embedding=nn.Embedding(max_langth, hidden_dim)
        
        self.layers=nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio) for _ in range(n_layers)])
        self.dropout=nn.Dropout(dropout_ratio)
        self.scale=torch.sqrt(torch.FloatTensor([hidden_dim]))
        
    def forward(self, src, src_mask):
        batch_size=src.shape[0]
        src_len=src.shape[1]
        
        pos=torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1) #add one dimension
        src=self.dropout((self.tok_embedding(src)*self.scale)+self.pos_embedding(pos))
        
        for layer in self.layers:
            src=layer(src, src_mask)
        return src